In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
import pandas as pd
spark = SparkSession.builder.appName("test").getOrCreate()
df = spark.read.csv('A_lvr_land_A.csv', header=True, inferSchema=True)
df = df.union(spark.read.csv('B_lvr_land_A.csv', header=True, inferSchema=True)) 
df = df.union(spark.read.csv('E_lvr_land_A.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('F_lvr_land_A.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('H_lvr_land_A.csv', header=True, inferSchema=True))
#篩選資料
df = df.filter(df['主要用途'].contains('住家用')).filter(df['建物型態'].contains('住宅大樓')).filter(~df['總樓層數'].isin(['一層','二層','三層','四層','五層','六層','七層','八層','九層','十層','十一層','十二層']))
df = df.sort(desc('交易年月日'))

In [ ]:
#把df丟進迴圈執行
def chinese_to_date(x): # 將民國轉換成西元
    return str(int(x)+19110000)[0:4]+'-' + str(int(x)+19110000)[4:6] + '-' + str(int(x)+19110000)[6:8]
for i in df.collect():
    print('{city:"台北市","time_slots": [{'+'"data":"'+str(chinese_to_date(i['交易年月日']))+'"',i['鄉鎮市區'],i['建物型態'])
    print('\n')


In [ ]:
print(df1.count())

In [ ]:
# df.printSchema()
# df.dtypes
df.createOrReplaceTempView('df')
spark.sql("select * from df ").show(10)

